In [1]:
import pandas as pd
import numpy as np

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

pd.set_option('display.max_columns', None)

In [2]:
manga = pd.read_csv('manga.csv')
manga.head()

,manga_id,title,type,score,scored_by,status,volumes,chapters,start_date,end_date,members,favorites,sfw,approved,created_at_before,updated_at,real_start_date,real_end_date,genres,themes,demographics,authors,serializations,synopsis,background,main_picture,url,title_english,title_japanese,title_synonyms,jikan
0,2,Berserk,manga,9.45,268737,currently_publishing,NaN,NaN,1989-08-25,NaN,551266,103820,True,True,2007-07-17 20:14:45+00:00,2022-06-23 08:30:44+00:00,1989-08-25,NaN,"['Action', 'Adventure', 'Award Winning', 'Dram...","['Gore', 'Military', 'Mythology', 'Psychologic...",['Seinen'],"[{'id': 1868, 'first_name': 'Kentarou', 'last_...",['Young Animal'],"Guts, a former mercenary now known as the ""Bla...",Berserk won the Award for Excellence at the si...,https://cdn.myanimelist.net/images/manga/1/157...,https://myanimelist.net/manga/2/Berserk,Berserk,ベルセルク,['Berserk: The Prototype'],True
1,13,One Piece,manga,9.20,305917,currently_publishing,NaN,NaN,1997-07-22,NaN,501291,99526,True,True,2007-07-17 20:14:45+00:00,2022-04-18 04:48:35+00:00,1997-07-22,NaN,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],"[{'id': 1881, 'first_name': 'Eiichiro', 'last_...",['Shounen Jump (Weekly)'],"Gol D. Roger, a man referred to as the ""Pirate...",One Piece is the highest selling manga series ...,https://cdn.myanimelist.net/images/manga/2/253...,https://myanimelist.net/manga/13/One_Piece,One Piece,ONE PIECE,[],True
2,25,Fullmetal Alchemist,manga,9.05,143879,finished,27.0,116.0,2001-07-12,2010-09-11,264571,28519,True,True,2007-07-17 20:14:45+00:00,2022-06-13 18:46:09+00:00,2001-07-12,2010-09-11,"['Action', 'Adventure', 'Award Winning', 'Dram...",['Military'],['Shounen'],"[{'id': 1874, 'first_name': 'Hiromu', 'last_na...",['Shounen Gangan'],Alchemists are knowledgeable and naturally tal...,Hagane no Renkinjutsushi won the 49th Shogakuk...,https://cdn.myanimelist.net/images/manga/3/243...,https://myanimelist.net/manga/25/Fullmetal_Alc...,Fullmetal Alchemist,鋼の錬金術師,"['Full Metal Alchemist', 'Hagane no Renkinjuts...",True
3,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,9.27,125482,finished,24.0,96.0,2004-01-19,2011-04-19,209123,35456,True,True,2007-10-07 08:14:20+00:00,2022-04-18 04:02:35+00:00,2004-01-19,2011-04-19,"['Action', 'Adventure', 'Horror', 'Mystery', '...",['Historical'],"['Seinen', 'Shounen']","[{'id': 2619, 'first_name': 'Hirohiko', 'last_...",['Ultra Jump'],"In the American Old West, the world's greatest...",JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,https://cdn.myanimelist.net/images/manga/3/179...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,"[""JoJo's Bizarre Adventure Part 7: Steel Ball ...",True
4,4632,Oyasumi Punpun,manga,9.03,141146,finished,13.0,147.0,2007-03-15,2013-11-02,354176,43168,True,True,2008-02-03 15:54:30+00:00,2022-04-18 04:47:05+00:00,2007-03-15,2013-11-02,"['Drama', 'Slice of Life']",['Psychological'],['Seinen'],"[{'id': 2836, 'first_name': 'Inio', 'last_name...",['Big Comic Spirits'],Punpun Onodera is a normal 11-year-old boy liv...,Oyasumi Punpun was originally serialized in Yo...,https://cdn.myanimelist.net/images/manga/3/164...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,Goodnight Punpun,おやすみプンプン,[],True


In [3]:
manga=manga.dropna(subset=['title', 'score', 'main_picture'])

In [4]:
manga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21909 entries, 0 to 21909
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manga_id           21909 non-null  int64  
 1   title              21909 non-null  object 
 2   type               21909 non-null  object 
 3   score              21909 non-null  float64
 4   scored_by          21909 non-null  int64  
 5   status             21909 non-null  object 
 6   volumes            16810 non-null  float64
 7   chapters           18011 non-null  float64
 8   start_date         21024 non-null  object 
 9   end_date           17490 non-null  object 
 10  members            21909 non-null  int64  
 11  favorites          21909 non-null  int64  
 12  sfw                21909 non-null  bool   
 13  approved           21909 non-null  bool   
 14  created_at_before  21909 non-null  object 
 15  updated_at         21699 non-null  object 
 16  real_start_date    210

# Menghitung Score

In [5]:
C = manga['score'].mean()

In [6]:
m = manga['scored_by'].quantile(0.75)

In [7]:
manga = manga.loc[manga['scored_by'] >= m]
manga.shape

(5478, 31)

In [8]:
def weighted_rating(x, m=m, C=C):
    v = x['scored_by']
    R = x['score']
    # Calculation based on the IMDB formula
    return np.round(((v/(v+m) * R) + (m/(m+v) * C)), 2)

In [9]:
manga['score_'] = manga.apply(weighted_rating, axis=1)

In [10]:
manga.drop(['score','scored_by', 'members', 'favorites', 'title_synonyms'], axis=1, inplace=True)

In [11]:
manga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5478 entries, 0 to 13993
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manga_id           5478 non-null   int64  
 1   title              5478 non-null   object 
 2   type               5478 non-null   object 
 3   status             5478 non-null   object 
 4   volumes            4281 non-null   float64
 5   chapters           4525 non-null   float64
 6   start_date         5432 non-null   object 
 7   end_date           4464 non-null   object 
 8   sfw                5478 non-null   bool   
 9   approved           5478 non-null   bool   
 10  created_at_before  5478 non-null   object 
 11  updated_at         5476 non-null   object 
 12  real_start_date    5432 non-null   object 
 13  real_end_date      4464 non-null   object 
 14  genres             5478 non-null   object 
 15  themes             5478 non-null   object 
 16  demographics       5478

In [12]:
manga.isna().sum()

manga_id                0
title                   0
type                    0
status                  0
volumes              1197
chapters              953
start_date             46
end_date             1014
sfw                     0
approved                0
created_at_before       0
updated_at              2
real_start_date        46
real_end_date        1014
genres                  0
themes                  0
demographics            0
authors                 0
serializations          0
synopsis               57
background           3066
main_picture            0
url                     0
title_english        2847
title_japanese         36
jikan                   0
score_                  0
dtype: int64

---

In [13]:
# cek data duplikat
manga.duplicated().any()

False

---

In [14]:
list_col = ['genres', 'themes', 'demographics']
mapping = [("[", ""), ("]", ""), ("'", ""), ('"', '')]
for i, j in mapping:
    for k in list_col:
        manga[k] = manga[k].apply(lambda x: x.replace(i, j))

In [15]:
manga['authors'][18]

"[{'id': 7108, 'first_name': 'Kenji', 'last_name': 'Inoue', 'role': 'Story'}, {'id': 16441, 'first_name': 'Kimitake', 'last_name': 'Yoshioka', 'role': 'Art'}]"

In [16]:
manga['authors'] = manga['authors'].apply(eval)

In [17]:
def convert_(list_):
    authors=''
    if len(list_) == 1:
        dict_ = list_[0]
        authors = authors + dict_['last_name'] + ', ' + dict_['first_name'] + ' (' + dict_['role'] + ')'
    else:
        for i in range(len(list_)):
            dict_ = list_[i]
            authors = authors + dict_['last_name'] + ', ' + dict_['first_name'] + ' (' + dict_['role'] + ')'
            if i < (len(list_)-1):
                authors = authors + ', '
    return authors

In [18]:
manga['authors'] = manga['authors'].apply(convert_)

In [19]:
manga['authors'][18]

'Inoue, Kenji (Story), Yoshioka, Kimitake (Art)'

In [20]:
manga_soup = manga[['title', 'genres', 'themes', 'demographics', 'authors', 'sfw']].reset_index(drop=True)
manga_soup

,title,genres,themes,demographics,authors,sfw
0,Berserk,"Action, Adventure, Award Winning, Drama, Fanta...","Gore, Military, Mythology, Psychological",Seinen,"Miura, Kentarou (Story & Art), Studio Gaga, (...",True
1,One Piece,"Action, Adventure, Fantasy",,Shounen,"Oda, Eiichiro (Story & Art)",True
2,Fullmetal Alchemist,"Action, Adventure, Award Winning, Drama, Fantasy",Military,Shounen,"Arakawa, Hiromu (Story & Art)",True
3,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,"Action, Adventure, Horror, Mystery, Supernatural",Historical,"Seinen, Shounen","Araki, Hirohiko (Story & Art)",True
4,Oyasumi Punpun,"Drama, Slice of Life",Psychological,Seinen,"Asano, Inio (Story & Art)",True
...,...,...,...,...,...,...
5473,Bozebeats,"Action, Supernatural",,Shounen,"Hirano, Ryouji (Story & Art)",True
5474,"""Boku wa Senpai ni Josou wo Shiirareteimasu.""","Comedy, Ecchi",Crossdressing,,"Shono, Kotaro (Story & Art)",True
5475,Jigoku no Kisetsu: Gurolism Sengen,"Hentai, Horror",,,"Horihone, Saizou (Story & Art), Uziga, Waita (...",False
5476,F-Ken,"Comedy, Sports","Combat Sports, School",Shounen,"Matsui, Yuusei (Story & Art)",True


In [21]:
unwanted_str = [('Story & Art', ''), ('Story', ''), ('Art', ''), ('(', ''), (')', '')]
for i, j in unwanted_str:
    manga_soup['authors'] = manga_soup['authors'].str.replace(i, j)

In [22]:
mapping_soup = [(" ", ""), (",", " ")]
for i, j in mapping_soup:
    for k in ['genres', 'themes', 'demographics', 'authors']:
        manga_soup[k] = manga_soup[k].apply(lambda x: x.replace(i, j))

In [23]:
manga_soup.isna().sum()

title           0
genres          0
themes          0
demographics    0
authors         0
sfw             0
dtype: int64

In [24]:
manga_soup.isna().sum()

title           0
genres          0
themes          0
demographics    0
authors         0
sfw             0
dtype: int64

---

In [25]:
manga_soup['soup'] = manga_soup.title + ' ' + manga_soup.genres + ' ' + manga_soup.themes + ' ' + manga_soup.demographics + ' ' + manga_soup.authors

In [26]:
manga_soup=manga_soup['soup']

In [27]:
for i in manga_soup:
     manga_soup.loc[manga_soup==i] = " ".join(i.split())

In [28]:
manga_soup = manga_soup.apply(lambda x: x.lower())
manga_soup

0       berserk action adventure awardwinning drama fa...
1       one piece action adventure fantasy shounen oda...
2       fullmetal alchemist action adventure awardwinn...
3       jojo no kimyou na bouken part 7: steel ball ru...
4       oyasumi punpun drama sliceoflife psychological...
                              ...                        
5473    bozebeats action supernatural shounen hirano r...
5474    "boku wa senpai ni josou wo shiirareteimasu." ...
5475    jigoku no kisetsu: gurolism sengen hentai horr...
5476    f-ken comedy sports combatsports school shoune...
5477            yoake mae ecchi romance josei yamada kana
Name: soup, Length: 5478, dtype: object

In [29]:
manga_soup.to_csv('manga_soup_genres_authors_based.csv', index=False)

In [30]:
manga_soup = pd.read_csv('manga_soup_genres_authors_based.csv')

In [31]:
manga_soup

,soup
0,berserk action adventure awardwinning drama fa...
1,one piece action adventure fantasy shounen oda...
2,fullmetal alchemist action adventure awardwinn...
3,jojo no kimyou na bouken part 7: steel ball ru...
4,oyasumi punpun drama sliceoflife psychological...
...,...
5473,bozebeats action supernatural shounen hirano r...
5474,"""boku wa senpai ni josou wo shiirareteimasu."" ..."
5475,jigoku no kisetsu: gurolism sengen hentai horr...
5476,f-ken comedy sports combatsports school shoune...


---

## Manga Display

In [32]:
manga.head()

,manga_id,title,type,status,volumes,chapters,start_date,end_date,sfw,approved,created_at_before,updated_at,real_start_date,real_end_date,genres,themes,demographics,authors,serializations,synopsis,background,main_picture,url,title_english,title_japanese,jikan,score_
0,2,Berserk,manga,currently_publishing,NaN,NaN,1989-08-25,NaN,True,True,2007-07-17 20:14:45+00:00,2022-06-23 08:30:44+00:00,1989-08-25,NaN,"Action, Adventure, Award Winning, Drama, Fanta...","Gore, Military, Mythology, Psychological",Seinen,"Miura, Kentarou (Story & Art), Studio Gaga, (...",['Young Animal'],"Guts, a former mercenary now known as the ""Bla...",Berserk won the Award for Excellence at the si...,https://cdn.myanimelist.net/images/manga/1/157...,https://myanimelist.net/manga/2/Berserk,Berserk,ベルセルク,True,9.44
1,13,One Piece,manga,currently_publishing,NaN,NaN,1997-07-22,NaN,True,True,2007-07-17 20:14:45+00:00,2022-04-18 04:48:35+00:00,1997-07-22,NaN,"Action, Adventure, Fantasy",,Shounen,"Oda, Eiichiro (Story & Art)",['Shounen Jump (Weekly)'],"Gol D. Roger, a man referred to as the ""Pirate...",One Piece is the highest selling manga series ...,https://cdn.myanimelist.net/images/manga/2/253...,https://myanimelist.net/manga/13/One_Piece,One Piece,ONE PIECE,True,9.19
2,25,Fullmetal Alchemist,manga,finished,27.0,116.0,2001-07-12,2010-09-11,True,True,2007-07-17 20:14:45+00:00,2022-06-13 18:46:09+00:00,2001-07-12,2010-09-11,"Action, Adventure, Award Winning, Drama, Fantasy",Military,Shounen,"Arakawa, Hiromu (Story & Art)",['Shounen Gangan'],Alchemists are knowledgeable and naturally tal...,Hagane no Renkinjutsushi won the 49th Shogakuk...,https://cdn.myanimelist.net/images/manga/3/243...,https://myanimelist.net/manga/25/Fullmetal_Alc...,Fullmetal Alchemist,鋼の錬金術師,True,9.03
3,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,finished,24.0,96.0,2004-01-19,2011-04-19,True,True,2007-10-07 08:14:20+00:00,2022-04-18 04:02:35+00:00,2004-01-19,2011-04-19,"Action, Adventure, Horror, Mystery, Supernatural",Historical,"Seinen, Shounen","Araki, Hirohiko (Story & Art)",['Ultra Jump'],"In the American Old West, the world's greatest...",JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,https://cdn.myanimelist.net/images/manga/3/179...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,True,9.25
4,4632,Oyasumi Punpun,manga,finished,13.0,147.0,2007-03-15,2013-11-02,True,True,2008-02-03 15:54:30+00:00,2022-04-18 04:47:05+00:00,2007-03-15,2013-11-02,"Drama, Slice of Life",Psychological,Seinen,"Asano, Inio (Story & Art)",['Big Comic Spirits'],Punpun Onodera is a normal 11-year-old boy liv...,Oyasumi Punpun was originally serialized in Yo...,https://cdn.myanimelist.net/images/manga/3/164...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,Goodnight Punpun,おやすみプンプン,True,9.01


In [33]:
manga.drop(['manga_id', 'created_at_before', 'updated_at', 'real_start_date', 'real_end_date', 'serializations', 'url'], axis=1, inplace=True)

In [34]:
manga.isna().sum()

title                0
type                 0
status               0
volumes           1197
chapters           953
start_date          46
end_date          1014
sfw                  0
approved             0
genres               0
themes               0
demographics         0
authors              0
synopsis            57
background        3066
main_picture         0
title_english     2847
title_japanese      36
jikan                0
score_               0
dtype: int64

In [35]:
list_unknown = ['volumes', 'chapters', 'start_date']
manga[list_unknown] = manga[list_unknown].fillna('Unknown')

col =['genres', 'authors']
for i in col:
    manga[col] = manga[col].apply(lambda x: x.replace('', 'Unknown'))

manga['end_date'] = manga['end_date'].fillna('?')

In [36]:
list_ = ['title_english', 'title_japanese']
manga[list_] = manga[list_].fillna('-')
manga['synopsis'] = manga['synopsis'].fillna('No synopsis for this title')

In [37]:
for i in ['status', 'type']:
    manga[i] = manga[i].apply(lambda x: x.replace('_',' '))

In [38]:
manga.isna().sum()

title                0
type                 0
status               0
volumes              0
chapters             0
start_date           0
end_date             0
sfw                  0
approved             0
genres               0
themes               0
demographics         0
authors              0
synopsis             0
background        3066
main_picture         0
title_english        0
title_japanese       0
jikan                0
score_               0
dtype: int64

In [39]:
manga

,title,type,status,volumes,chapters,start_date,end_date,sfw,approved,genres,themes,demographics,authors,synopsis,background,main_picture,title_english,title_japanese,jikan,score_
0,Berserk,manga,currently publishing,Unknown,Unknown,1989-08-25,?,True,True,"Action, Adventure, Award Winning, Drama, Fanta...","Gore, Military, Mythology, Psychological",Seinen,"Miura, Kentarou (Story & Art), Studio Gaga, (...","Guts, a former mercenary now known as the ""Bla...",Berserk won the Award for Excellence at the si...,https://cdn.myanimelist.net/images/manga/1/157...,Berserk,ベルセルク,True,9.44
1,One Piece,manga,currently publishing,Unknown,Unknown,1997-07-22,?,True,True,"Action, Adventure, Fantasy",,Shounen,"Oda, Eiichiro (Story & Art)","Gol D. Roger, a man referred to as the ""Pirate...",One Piece is the highest selling manga series ...,https://cdn.myanimelist.net/images/manga/2/253...,One Piece,ONE PIECE,True,9.19
2,Fullmetal Alchemist,manga,finished,27.0,116.0,2001-07-12,2010-09-11,True,True,"Action, Adventure, Award Winning, Drama, Fantasy",Military,Shounen,"Arakawa, Hiromu (Story & Art)",Alchemists are knowledgeable and naturally tal...,Hagane no Renkinjutsushi won the 49th Shogakuk...,https://cdn.myanimelist.net/images/manga/3/243...,Fullmetal Alchemist,鋼の錬金術師,True,9.03
3,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,finished,24.0,96.0,2004-01-19,2011-04-19,True,True,"Action, Adventure, Horror, Mystery, Supernatural",Historical,"Seinen, Shounen","Araki, Hirohiko (Story & Art)","In the American Old West, the world's greatest...",JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,https://cdn.myanimelist.net/images/manga/3/179...,-,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,True,9.25
4,Oyasumi Punpun,manga,finished,13.0,147.0,2007-03-15,2013-11-02,True,True,"Drama, Slice of Life",Psychological,Seinen,"Asano, Inio (Story & Art)",Punpun Onodera is a normal 11-year-old boy liv...,Oyasumi Punpun was originally serialized in Yo...,https://cdn.myanimelist.net/images/manga/3/164...,Goodnight Punpun,おやすみプンプン,True,9.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13773,Bozebeats,manga,finished,2.0,15.0,2018-01-15,2018-04-16,True,True,"Action, Supernatural",,Shounen,"Hirano, Ryouji (Story & Art)",Religious monks adapted to modern technology a...,"As a part of the JUMP START initiative, the fi...",https://cdn.myanimelist.net/images/manga/1/209...,-,BOZEBEATS,True,6.40
13796,"""Boku wa Senpai ni Josou wo Shiirareteimasu.""",manga,finished,1.0,6.0,2012-04-25,2013-12-25,True,True,"Comedy, Ecchi",Crossdressing,,"Shono, Kotaro (Story & Art)",Wataru starts his high school life by getting ...,NaN,https://cdn.myanimelist.net/images/manga/1/188...,-,「僕は先輩に女装を強いられています。」,True,6.43
13894,Jigoku no Kisetsu: Gurolism Sengen,manga,finished,1.0,10.0,2003-12-31,2003-12-31,False,True,"Hentai, Horror",,,"Horihone, Saizou (Story & Art), Uziga, Waita (...",A compilation of short stories by various ero-...,NaN,https://cdn.myanimelist.net/images/manga/2/492...,-,地獄の季節 ―グロリズム宣言―,True,6.30
13980,F-Ken,one shot,finished,Unknown,1.0,2019-08-05,2019-08-05,True,True,"Comedy, Sports","Combat Sports, School",Shounen,"Matsui, Yuusei (Story & Art)",Fuji's high school kendo team has an embarrass...,F-Ken was simulpub in English through MANGA Pl...,https://cdn.myanimelist.net/images/manga/3/220...,-,Fけん,True,6.02


In [40]:
manga.to_csv('manga_display.csv', index=False)

In [41]:
manga = pd.read_csv('manga_display.csv')

----